In [3]:
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
import uuid

In [6]:
import json
import chromadb
import transformers
from chromadb.utils import embedding_functions
# Initialize the chromadb directory, and client.
db = chromadb.PersistentClient(path="./chroma_db_mpnet")
embedder = embedding_functions.SentenceTransformerEmbeddingFunction(
     model_name= 'all-mpnet-base-v2'
)
collection = db.get_or_create_collection(
    name=f"chroma_db_mpnet",
    embedding_function= embedder
)

c:\Users\Trung\anaconda3\envs\master_torch_gpu\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.embeddings_db
collection = db.corpus_embeddings

In [11]:
def add_documents_in_batches(collection, corpus, embeddings, batch_size=10000):
    total_added = 0
    for i in range(0, len(corpus), batch_size):
        batch_ids = []
        batch_texts = []
        batch_embeddings = []

        for item, embedding in zip(corpus[i:i+batch_size], embeddings[i:i+batch_size]):
            if item[0].strip():  # Check if the text content is not empty
                batch_ids.append(str(uuid.uuid4()))
                batch_texts.append(item[0])
                batch_embeddings.append(embedding.tolist())

        if batch_ids:  # Only add if there are valid documents in the batch
            collection.add(
                ids=batch_ids,
                documents=batch_texts,
                embeddings=batch_embeddings
            )
            total_added += len(batch_ids)
            print(f"Added batch of {len(batch_ids)} documents. Total added: {total_added}")

    return total_added

# Add documents in batches
total_documents_added = add_documents_in_batches(collection, retrieved_corpus, retrieved_embeddings)

print(f"Total documents added to ChromaDB: {total_documents_added}")

Added batch of 10000 documents. Total added: 10000
Added batch of 9989 documents. Total added: 19989
Added batch of 10000 documents. Total added: 29989
Added batch of 9999 documents. Total added: 39988
Added batch of 10000 documents. Total added: 49988
Added batch of 10000 documents. Total added: 59988
Added batch of 10000 documents. Total added: 69988
Added batch of 10000 documents. Total added: 79988
Added batch of 10000 documents. Total added: 89988
Added batch of 10000 documents. Total added: 99988
Added batch of 10000 documents. Total added: 109988
Added batch of 2591 documents. Total added: 112579
Total documents added to ChromaDB: 112579


In [8]:
results = collection.query(
    query_texts=["Smart Skincare Solutions"], # Chroma will embed this for you
    n_results=10 # how many results to return
)
print(results)
print("Top 10 similar documents:")
for i, (doc_id, distance, document) in enumerate(zip(results['ids'][0], results['distances'][0], results['documents'][0]), 1):
    print(f"   Distance: {distance:.4f}")
    print(f"   Document: {document}")
    print()

{'ids': [['3db703ca-031a-4cd4-9951-04be1b5cdf1f', '20d763b1-3f43-4237-a9c3-1986391c51cc', '2c194800-5291-45e5-bdb5-e634a7684d91', 'bfd16922-e518-4e72-be19-a348afb6d99e', '6e648497-bff2-4884-b026-e82243bfbe54', '33668e6c-4b77-4b4d-87dc-bd6e5f9378cb', '07436e67-c471-4ed1-91ef-4976d69d1093', '24499196-d7b5-432a-a755-44c27fb42fc7', '44850c45-af01-435b-9d0d-3c0fcc26b8ec', 'd80de646-9b9a-4095-aab5-b9f6cb0668fa']], 'distances': [[0.6799821853637695, 0.7166837453842163, 0.7207689881324768, 0.7686786651611328, 0.7722411155700684, 0.7796213030815125, 0.7815015912055969, 0.7827130556106567, 0.7850625514984131, 0.7893216609954834]], 'metadatas': [[None, None, None, None, None, None, None, None, None, None]], 'embeddings': None, 'documents': [['KINGDOMCARES Hot Mist Facial Steamer Warm Nano Ionic Moisturizing Face Humidifier Personal Home Sauna SPA Unclogs Pores Blackheads Acne Comedone Removal Facial Hydration System Nano Sprayer Pink', 'Anti-Aging Armour Super Smart Skin-Perfecting Beauty Fluid S